# IMDB Dataset - Create Weak Supervision Sources

This notebook shows how to create labeling functions on the IMDB Movie Review dataset.

This dataset has gold labels. These labels are just there for evaluation purposes. The idea of using weak supervision and especially knodle is that you don't have a dataset which is purely labeled with strong supervision (manual) and instead label it with weak supervision.

First, we load the dataset from kaggle. Then, we will look at certain keywords within both sentiments and find good matching keywords which will act as a weak supervision source. Finally, the keywords on a basic majority vote model.

## Imports

Lets make some basic imports 

In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, CountVectorizer
from bs4 import BeautifulSoup
from snorkel.labeling import LabelingFunction, PandasLFApplier, filter_unlabeled_dataframe, LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
import numpy as np 
from tqdm import tqdm

In [2]:
# Init
tqdm.pandas()
pd.set_option('display.max_colwidth', -1)

/Users/sandro/.pyenv/versions/3.7.1/envs/imdbTut/lib/python3.7/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/Users/sandro/.pyenv/versions/3.7.1/envs/imdbTut/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
# Constants
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1
COLUMN_WITH_TEXT = "reviews_preprocessed"

## Download the raw dataset

Now we download the dataset we need. For that you need to have the kaggle-cli installed and configured with your API key. Please have a look at the official [documentation](https://github.com/Kaggle/kaggle-api) for further instructions.

In [4]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!tar -xvf imdb-dataset-of-50k-movie-reviews.zip
!rm imdb-dataset-of-50k-movie-reviews.zip

/Users/sandro/.pyenv/versions/3.7.1/envs/imdbTut/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.0) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
100%|██████████████████████████████████████| 25.7M/25.7M [00:02<00:00, 9.98MB/s]
100%|██████████████████████████████████████| 25.7M/25.7M [00:02<00:00, 10.9MB/s]
x IMDB Dataset.csv


## Preview dataset

After downloading and unpacking the dataset we can have a first look at it and work with it.

In [5]:
imdb_dataset_raw = pd.read_csv('IMDB Dataset.csv')

In [6]:
imdb_dataset_raw.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

In [7]:
imdb_dataset_raw.groupby('sentiment').count()


,review
sentiment,
negative,25000
positive,25000


In [8]:
imdb_dataset_raw.isna().sum()

review       0
sentiment    0
dtype: int64

## Preprocess dataset

Now lets take some basic preprocessing steps

### Remove Stopwords

We begin by removing all common stop words. We use `scikit-learn`'s stopwords that we don't install to many packages.

In [9]:
imdb_dataset_raw['reviews_preprocessed'] = imdb_dataset_raw['review'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in (ENGLISH_STOP_WORDS)]))

In [10]:
imdb_dataset_raw.head()

,review,sentiment,reviews_preprocessed
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive,"One reviewers mentioned watching just 1 Oz episode you'll hooked. They right, exactly happened me.<br /><br />The thing struck Oz brutality unflinching scenes violence, set right word GO. Trust me, faint hearted timid. This pulls punches regards drugs, sex violence. Its hardcore, classic use word.<br /><br />It called OZ nickname given Oswald Maximum Security State Penitentary. It focuses mainly Emerald City, experimental section prison cells glass fronts face inwards, privacy high agenda. Em City home many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish more....so scuffles, death stares, dodgy dealings shady agreements far away.<br /><br />I say main appeal fact goes shows wouldn't dare. Forget pretty pictures painted mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The episode I saw struck nasty surreal, I couldn't say I ready it, I watched more, I developed taste Oz, got accustomed high levels graphic violence. Not just violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order away it, mannered, middle class inmates turned prison bitches lack street skills prison experience) Watching Oz, comfortable uncomfortable viewing....thats touch darker side."
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell'

### Remove HTML Tags

The dataset contains many HTML tags. We'll remove them

In [11]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

imdb_dataset_raw['reviews_preprocessed'] = imdb_dataset_raw['reviews_preprocessed'].apply(
    lambda x: strip_html(x))

In [12]:
imdb_dataset_raw['reviews_preprocessed'].head()

0    One reviewers mentioned watching just 1 Oz episode you'll hooked. They right, exactly happened me.The thing struck Oz brutality unflinching scenes violence, set right word GO. Trust me, faint hearted timid. This pulls punches regards drugs, sex violence. Its hardcore, classic use word.It called OZ nickname given Oswald Maximum Security State Penitentary. It focuses mainly Emerald City, experimental section prison cells glass fronts face inwards, privacy high agenda. Em City home many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish more....so scuffles, death stares, dodgy dealings shady agreements far away.I say main appeal fact goes shows wouldn't dare. Forget pretty pictures painted mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The episode I saw struck nasty surreal, I couldn't say I ready it, I watched more, I developed taste Oz, got accustomed high levels graphic violence. Not just violence, injustice (crooked guards who'll sold n

## Keywords

For weak supervision sources we use sentiment keyword lists for positive and negative words.
https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html

In [16]:
from minio import Minio


In [24]:
f = open("/Users/sandro/Downloads/opinion-lexicon-English/positive-words.txt", "r")
positive_keywords = f.readlines()

In [70]:
positive_keywords = pd.read_csv('/Users/sandro/Downloads/opinion-lexicon-English/positive-words.txt', sep=" ", header=None, error_bad_lines=False)
positive_keywords.columns = ["keywords"]
positive_keywords = positive_keywords.drop_duplicates("keywords")

In [69]:
negative_keywords = pd.read_csv('/Users/sandro/Downloads/opinion-lexicon-English/negative-words.txt',
                                sep=" ", header=None, error_bad_lines=False,  encoding='latin-1')
print("negative keywords orig shape: ", negative_keywords.shape)
negative_keywords.columns = ["keywords"]
negative_keywords = negative_keywords.drop_duplicates("keywords")
print("negative keywords shape after droppping duplicates: ", negative_keywords.shape)

negative keywords orig shape:  (4783, 1)
negative keywords shape after droppping duplicates:  (4783, 1)


In [82]:
positive_keywords

,keywords
0,a+
1,abound
2,abounds
3,abundance
4,abundant
...,...
2001,youthful
2002,zeal
2003,zenith
2004,zest


In [85]:
all_keywords = pd.DataFrame({'keyword':positive_keywords.keywords, 'sentiment':'positive'}, index=positive_keywords.index)
all_keywords.head()

,keyword,sentiment
0,a+,positive
1,abound,positive
2,abounds,positive
3,abundance,positive
4,abundant,positive


In [89]:
negative_keywords = pd.DataFrame({'keyword':negative_keywords.keywords, 'sentiment':'negative'}, index=negative_keywords.index)


AttributeError: 'DataFrame' object has no attribute 'keywords'

In [91]:
all_keywords = pd.concat([all_keywords, negative_keywords])

In [92]:
all_keywords.shape

(6789, 2)

In [95]:
all_keywords.drop_duplicates('keyword').shape

(6786, 2)

In [96]:
all_keywords.drop_duplicates('keyword',inplace=True)

## Labeling Functions

Now we start to build labeling functions with Snorkel with these keywords and check the coverage.

This is an iterative process of course so we surely have to add more keywords and regulary expressions ;-) 

In [97]:
def keyword_lookup(x, keyword, label):
    return label if keyword in x[COLUMN_WITH_TEXT].lower() else ABSTAIN


In [98]:
def make_keyword_lf(keyword: str, label: str) -> LabelingFunction:
    """
    Creates labeling function based on keyword.
    Args:
        keywords:
        label:

    Returns:

    """
    return LabelingFunction(
        name=f"keyword_{keyword}",
        f=keyword_lookup,
        resources=dict(keyword=keyword, label=label),
    )

In [99]:
def create_labeling_functions(keywords: pd.DataFrame) -> np.ndarray:
    """
    Create Labeling Functions based on the columns keyword and regex. Appends column lf to df.

    Args:
        keywords: DataFrame with processed keywords

    Returns:
        All labeling functions. 1d Array with shape: (number_of_lfs x 1)
    """
    keywords = keywords.assign(lf=keywords.progress_apply(
        lambda x:make_keyword_lf(x.keyword, x.label_id), axis=1
    ))
    lfs = keywords.lf.values
    return lfs

In [100]:
def make_keyword_df(positive_keywords: [str], negative_keywords: [str]) -> pd.DataFrame:
    positive = pd.DataFrame(
        pd.Series({x: 'positive' for (x) in positive_keywords}, name="label")).reset_index().rename(columns={"index":"keyword"})
    
    negative = pd.DataFrame(
        pd.Series({x: 'negative' for (x) in negative_keywords}, name="label")).reset_index().rename(columns={"index":"keyword"})
    keywords = positive.append(negative)
    
    assert len(positive) + len(negative) == len(keywords), "Shapes doesn't match"
    
    keywords.loc[keywords.label == 'positive', 'label_id'] = int(POSITIVE)
    keywords.loc[keywords.label == 'negative', 'label_id'] = int(NEGATIVE)
    keywords.reset_index(inplace=True, drop=True)
    return keywords


In [101]:
keywords = make_keyword_df(all_keywords.loc[all_keywords.sentiment == 'positive', 'keyword'].values,
                           all_keywords.loc[all_keywords.sentiment == 'negative', 'keyword'].values,)

In [102]:
keywords

,keyword,label,label_id
0,a+,positive,1.0
1,abound,positive,1.0
2,abounds,positive,1.0
3,abundance,positive,1.0
4,abundant,positive,1.0
...,...,...,...
6781,zaps,negative,0.0
6782,zealot,negative,0.0
6783,zealous,negative,0.0
6784,zealously,negative,0.0


In [103]:
labeling_functions = create_labeling_functions(keywords)

100%|██████████| 6786/6786 [00:00<00:00, 35806.49it/s]


In [104]:
labeling_functions

array([LabelingFunction keyword_a+, Preprocessors: [],
       LabelingFunction keyword_abound, Preprocessors: [],
       LabelingFunction keyword_abounds, Preprocessors: [], ...,
       LabelingFunction keyword_zealous, Preprocessors: [],
       LabelingFunction keyword_zealously, Preprocessors: [],
       LabelingFunction keyword_zombie, Preprocessors: []], dtype=object)

### Apply Labeling Functions

Now lets apply all labeling functions on our reviews and check some statistics.

In [105]:
applier = PandasLFApplier(lfs=labeling_functions)
applied_lfs = applier.apply(df=imdb_dataset_raw)

/Users/sandro/.pyenv/versions/3.7.1/envs/imdbTut/lib/python3.7/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 50000/50000 [34:44<00:00, 23.98it/s]


In [106]:
applied_lfs

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]])

Now we have a matrix with all labeling functions applied. This matrix has the shape $(instances \times labeling functions)$

In [107]:
print("Shape of applied labeling functions: ", applied_lfs.shape)
print("Number of reviews", len(imdb_dataset_raw))
print("Number of labeling functions", len(labeling_functions))

Shape of applied labeling functions:  (50000, 6786)
Number of reviews 50000
Number of labeling functions 6786


### Analysis

Now we can analyse some basic stats about our labeling functions. The main figures are:

- Coverage: How many labeling functions match at all
- Overlaps: How many labeling functions overlap with each other (e.g. awesome and amazing)
- Conflicts: How many labeling functions overlap and have different labels (e.g. awesome and bad)
- Correct: Correct LFs
- Incorrect: Incorrect Lfs

In [108]:
LFAnalysis(L=applied_lfs, lfs=labeling_functions).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
keyword_a+,0,[1],0.00102,0.00102,0.00102
keyword_abound,1,[1],0.00264,0.00264,0.00264
keyword_abounds,2,[1],0.00052,0.00052,0.00052
keyword_abundance,3,[1],0.00182,0.00182,0.00182
keyword_abundant,4,[1],0.00114,0.00114,0.00114
...,...,...,...,...,...
keyword_zaps,6781,[0],0.00012,0.00012,0.00012
keyword_zealot,6782,[0],0.00048,0.00048,0.00048
keyword_zealous,6783,[0],0.00092,0.00092,0.00092
keyword_zealously,6784,[0],0.00006,0.00006,0.00006


In [109]:
lf_analysis = LFAnalysis(L=applied_lfs, lfs=labeling_functions).lf_summary()

In [110]:
pd.DataFrame(lf_analysis.mean())

,0
j,3392.500000
Coverage,0.005002
Overlaps,0.005002
Conflicts,0.004997


In [111]:
pd.DataFrame(lf_analysis.median())

,0
j,3392.50000
Coverage,0.00052
Overlaps,0.00052
Conflicts,0.00052


Lets have a look at some basic examples

In [112]:
imdb_dataset_raw.iloc[applied_lfs[:, 1] == POSITIVE, :].sample(10, random_state=1).loc[:, ['review','sentiment']]

review  \
30200  The summary line above, spoken by James Cloud (Robert Preston) to his brother Tom (Robert Sterling) just about says it all. Jim, AKA Kid Wichita, has a way of making things happen, only trouble is, he usually leaves dead bodies where he's been. Not the sort of mentoring Tom envisions for younger brother Jeff, who likes what he sees in Jim, especially when defending their ranch against local Texas cattlemen.<br /><br />The opening credits state 'Introducing John Barrymore Jr. as the Younger Brother', in this his very first screen appearance. That seemed rather odd to me, particularly since he was addressed as Jeff almost immediately into the story. Approximately eighteen at the time of this movie, he bears a passing resemblance to Sean Penn. No stranger to personal and legal problems throughout his career as well as estrangement from his family, I was left wondering if his daughter Drew Barrymore might have ever seen this picture. I'm inclined to think not.<br /><br />On the subject of resemblances, I was also struck by the thought that the young Robert Sterling looked a bit like Roy Rogers early in his career. Knowing Sterling previously only from his role as George Kerby in the early 1950's TV series 'Topper', I thought he looked out of place in a Western, but that might just be me. His character becomes emboldened by his brother's resourcefulness at creating trouble, and provides some of the edginess to this not so typical story. Minor subplots abound, including the relationship rancher John Gall (John Litel) has with his son the Sheriff (who Kid Wichita kills), and the troubled marriage between Kathleen Boyce (Cathy Downs) and her husband Earl (who Kid Wichita kills). Chill Wills rounds out the main cast as one of Tom Cloud's hired hands, and figures in the somewhat predictable finale.<br /><br />What's not quite predictable is how things eventually wind up there, and for that reason, this Western earns points for following a less traveled, hence not quite as formulaic a plot as a lot of good brother/bad brother Westerns do. Combined with the eclectic casting of the principals, it's one I'd recommend, even if you have to endure some of the jump cuts and sloppy editing that I experienced with my copy.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## Transform rule matches

To work with knodle the dataset needs to be transformed into a binary array.

0 -> Rule match
1 -> Rule didn't match 

Furthermore, we need a matrix `mapping_rule_labels` which has a mapping of all rules to labels.

In [115]:
from knodle.data.labelling_fcts import transform_rule_class_matrix_to_z_t

In [118]:
rule_matches, mapping_rules_labels = transform_rule_class_matrix_to_z_t(applied_lfs)

In [119]:
rule_label_counts = np.matmul(applied_lfs, mapping_rules_labels)

### Majority Vote

Now we make a majority vote based on all rule matches. First we get the `rule_counts` by multiplying `rule_matches` with the `mapping_rules_labels`, then we divide it sumwise by the sum to get a probability value. In the end we counteract the divide with zero issue by setting all nan values to zero.

In [124]:
from knodle.trainer.utils.denoise import get_majority_vote_labels, get_majority_vote_probs

In [125]:
preds = get_majority_vote_probs(rule_matches, mapping_rules_labels)

/Users/sandro/repo/knodle/knodle/trainer/utils/denoise.py:67: RuntimeWarning: invalid value encountered in true_divide
  rule_counts_probs = rule_counts / rule_counts.sum(axis=1).reshape(-1, 1)


In [128]:
preds_label = np.argmax(preds, axis=1)

In [129]:
# Positive is a bit overrepresented
np.unique(preds_label, return_counts=True)

(array([0, 1]), array([38103, 11897]))

In [130]:
imdb_dataset_raw['label_id'] = imdb_dataset_raw.sentiment.map({'positive':POSITIVE, 'negative':NEGATIVE})

## Save Files


In [133]:
from joblib import dump
from scipy.sparse import csr_matrix

In [134]:
rule_matches_sparse = csr_matrix(rule_matches)

In [136]:
dump(rule_matches_sparse, 'rule_matches.lib')
dump(mapping_rules_labels, "mapping_rules_labels.lib")

['mapping_rules_labels.lib']

In [72]:
!ls

IMDB Dataset.csv           keywords.csv
Prepare_IMDB_Data.ipynb    mapping_rules_labels.lib
applied_lfs.lib            requirements.txt
imdb_data_preprocessed.csv tfidf.lib


In [137]:
keywords.to_csv('keywords.csv', index=None)


In [68]:
imdb_dataset_raw.to_csv('imdb_data_preprocessed.csv', index=None)

# Finish

Now, we have created a weak supervision dataset. Of course it is not perfect but it is something with which we can compare performances of different denoising methods with. :-) 